# 02 · Make Chronological Slices

Build temporal windows and persist member / non-member panels on Drive.

In [ ]:
# Persistent project setup on Drive
import os
import sys
from pathlib import Path

DRIVE_ROOT = Path('/content/drive')
try:
    from google.colab import drive  # type: ignore
    if not DRIVE_ROOT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if DRIVE_ROOT.exists():
    BASE_ROOT = (DRIVE_ROOT / 'MyDrive').resolve()
else:
    BASE_ROOT = Path.home().resolve()

PROJECT_ROOT = BASE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Clone the repo via 00_colab_setup.ipynb before running this notebook.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

os.chdir(PROJECT_ROOT)
print('PROJECT_ROOT:', PROJECT_ROOT)


In [ ]:
from dataclasses import asdict

import pandas as pd

from src.data.loaders import load_canonical
from src.data.slicing import SliceConfig, assign_temporal_slices, enforce_token_budget, build_member_panels

CANONICAL_PATH = ARTIFACTS_DIR / 'canonical_demo.parquet'
df = load_canonical(CANONICAL_PATH)
if df.empty:
    raise FileNotFoundError('Run notebook 01 to create the canonical dataset first.')
print(df.head())


In [ ]:
df = assign_temporal_slices(df, total_slices=8)
df = enforce_token_budget(df, tokens_per_slice=50_000)
df[['subject_id', 'slice_id', 'tokens_estimate']].head()

In [ ]:
slice_config = SliceConfig()
panels = build_member_panels(df, config=slice_config, artifact_dir=str(ARTIFACTS_DIR))
print(f'Persisted ID panels for {len(panels)} slices under {ARTIFACTS_DIR / "slice_*/ids"}')


⚠️ Review chronological splits carefully with credentialed timestamps before running attacks.